# Microsatellite Instability (MSI) Analysis Pipeline

This Jupyter notebook demonstrates a simplified and fully anonymized version of an MSI analysis pipeline used in a research lab setting. The pipeline detects microsatellite instability in cancer samples using a set of genomic loci. **All data shown is simulated or anonymized.**

## Overview of the Pipeline

- Load sample output files from MSI sensor or similar tools
- Count the number of unstable loci per sample
- Classify samples into `MSI-High`, `MSI-Low`, or `MSI-Stable`
- Summarize results for downstream reporting or research

## 🛠Code Execution

Below is the Python code used to implement the analysis pipeline. It includes directory scanning, parsing results, and classifying MSI status per sample.

## Output Summary

After processing, the notebook prints a summary table of MSI classifications and optionally generates histograms or plots for visual analysis.

## **Disclaimer

This notebook is a **sanitized** and **anonymized** demo. It does not contain any protected health information (PHI) or real patient data. Shared for educational and portfolio purposes only.

### MSI Pipeline Analysis
#### Gathers data from MSI pipeline runs and outputs tables/histograms/scatter plots

#### imports all necessary libraries and initiates loci_data for use in for-loop

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np

folders = os.listdir('project/MSI_Analyses')

#enter loci data and make loci_df
loci_data = {
    "Chromosome": [
        "chr4", "chr2", "chr14", "chr2", "chr11", "chr11", "chr11", 
        "chr11", "chr7", "chr17", "chr13", "chr4", "chr6", "chr9"
    ],
    "Start": [
        55598211, 47641559, 23652346, 95849361, 102193508, 108188266, 
        108114661, 108141955, 116409675, 29508819, 31722620, 153268227, 
        117725383, 135773000
    ],
    "Target_Loci": [
        "BAT25", "BAT26", "NR-21", "NR-24", "NR-27", "T13", "T15-1", 
        "T15-2", "T15-3", "T16", "HSPH1-T17", "A14", "A15", "A18"
    ]
}
# New data
new_loci = { "Chromosome": ["chr1", "chr2", "chr7", "chr11", "chr13", "chr17", "chr1", "chr3", "chr3", "chr6", "chr8", "chr10", "chr11", "chr12", "chr18"], "Start": [120053340, 225422600, 116409675, 108114661, 48954159, 29559061, 162736821, 10076008, 69988437, 51503597, 141754888, 43595836, 119144791, 112893675, 45395845],"Target_Loci": ["BAT-40", "CUL-22", "MET-15", "ATM-15", "RB1-13", "NF1-26", "DDR-11", "FANC-21", "MITF-14", "PKHD-18", "PTK-16", "RET-14", "CBL-17", "PTPN-17", "SMAD-18"] }
# Combine existing and new
loci_data["Chromosome"].extend(new_loci["Chromosome"])
loci_data["Start"].extend(new_loci["Start"])
loci_data["Target_Loci"].extend(new_loci["Target_Loci"])
loci_df = pd.DataFrame(loci_data)

#### Runs a loop for each of the 10 samples in the directory folder. 
#### Loads the data into dataFrames then produces filtered_df,result_df, scatter plot, and histograms for each loci. 
#### If part of the positive group of samples it will add to combined_df for later use

In [ ]:
#runs for each file
combined_df = pd.DataFrame()
end_codes = {'730','281','460','174','486','684','090'}


for folder_name in folders:
    
    #gather the directory and file id
    if "DRAGEN Analysis" in folder_name:
        file_id = folder_name.split()[0]
        dir = f"project/MSI_Analyses/{folder_name}/{file_id}"
        # 
    else:
        continue
    print(file_id)

    #load in data to dataFrames
    microsat_diff_df = pd.read_csv(f"{dir}/{file_id}.microsat_diffs.txt",delimiter='\t')
    microsat_tumor_df = pd.read_csv(f"{dir}/{file_id}.microsat_tumor.dist",delimiter='\t')
    
    
    
    # Rename columns
    microsat_diff_df.rename(columns={'#Chromosome':'Chromosome'},inplace=True)
    microsat_tumor_df.rename(columns={'#chromosome':'Chromosome', 'location':'Start','repeat_unit_bases':'RepeatUnit'},inplace=True)
    
   
    # merge data on chromosome, start,repeatUnit
    merged_df = pd.merge(microsat_diff_df, microsat_tumor_df, on=['Chromosome', 'Start', 'RepeatUnit'])
    # drop unnecessary columns
    merged_df.drop(['PassFilter', 'covered'], axis=1,inplace = True)


    # Filter values and sort in ascending order by PValue
    filtered_df = merged_df[(merged_df.Assessed == True) & (merged_df.Distance >= 0.15) & (merged_df.PValue <= 0.015)]
    filtered_df = filtered_df.sort_values(by='PValue', ascending=True)
    
    filtered_df.to_csv(f'{dir}/{file_id}_outputs/{file_id}_filtered.csv',index=False)
    
    
    
    # Export scatter plots of merged_df
    plt.figure(figsize=(10, 6))
    plt.scatter(filtered_df['Distance'], filtered_df['PValue'])
    plt.title('Scatter plot of Distance vs PValue')
    plt.xlabel('Distance')
    plt.ylabel('PValue')
    plt.savefig(f'{dir}/{file_id}_outputs/{file_id}_filtered_scatter.png')
    
    
    
    
    # merge with loci_df
    result_df = merged_df.merge(loci_df, on=['Chromosome', 'Start'], how='inner')
    
    
    # re-order result to match the loci_df
    result_df.set_index("Target_Loci",inplace=True)
    loci_ordered = loci_df["Target_Loci"]
    result_df = result_df.loc[loci_ordered,:]
    result_df = result_df[(result_df.Assessed == True)]
    
    
    # Reset the index to convert the index column to a normal column
    result_df = result_df.reset_index()
    result_df.to_csv(f'{dir}/{file_id}_outputs/{file_id}_final.csv',index=False)
    code = file_id.split('_')[-1][-3:]
    
    #End codes contains the positive sample codes
    if code in end_codes:
        combined_df = pd.concat([combined_df,filtered_df],ignore_index=True)
             
    # Export Histograms for each loci
    msi_loci = ["BAT25", "BAT26", "NR-21", "NR-24", "NR-27"]

    # Load combined_df for normals distribution
    combined_df = pd.read_csv('/data/project/MSI_Normals_Panel/analysis_files/normals_dist_sum.csv')
    num_samples = len(os.listdir(dir))

    num_loci = len(msi_loci)
    col = 2
    row = math.ceil(num_loci / col)
    fig, axes = plt.subplots(row, col, figsize=(10 * col, 6 * row))
    axes = axes.flatten()

    # Overlayed histogram outputs
    for i, marker in enumerate(msi_loci):
        # Filter the DataFrame for the current marker
        dist_string = result_df[result_df['Target_Loci'] == marker]['length_distribution'].values[0]
        dist_list = list(map(int, dist_string.split(',')))

        normals_dist_string = combined_df[combined_df.Target_Loci == marker]['length_distribution'].values[0]
        normals_dist_list = np.array(list(map(int, normals_dist_string.split(',')))) / num_samples

        # Set the positions for Tumor and Normal bars
        bar_width = 0.4
        x_positions = np.arange(len(dist_list))

        # Plot Tumor distribution
        axes[i].bar(x_positions - bar_width / 2, dist_list, width=bar_width, alpha=0.5, label='Tumor', color='red')

        # Plot Normal distribution
        axes[i].bar(x_positions + bar_width / 2, normals_dist_list, width=bar_width, alpha=0.5, label='Normal', color='blue')
        axes[i].set_xlabel("Additional Base Pairs (bp)")
        axes[i].set_ylabel("Read Counts")
        axes[i].set_title(f'MSI Distribution at {marker} Locus')
        axes[i].grid(True)
        axes[i].legend()

    # Format and save output
    plt.tight_layout()
    output_filename = f'{dir}/{file_id}_outputs/loci_histograms.png'
    plt.savefig(output_filename)
    plt.close(fig)
    
    
    
print('----------DONE-----------')

#### Uses combined_df to generate a table of the common loci and their distance, repeatUnit,Start location, Chromosome, and reference_allele for each. Threshold of 6 or 7 can be changed. 

In [ ]:
key_columns = ['Chromosome', 'Start', 'RepeatUnit']
grouped = combined_df.groupby(key_columns).size().reset_index(name='counts')
threshold = 7
common_groups = grouped[grouped['counts'] >= threshold]
result_df = pd.merge(combined_df, common_groups[key_columns], on=key_columns, how='inner')
result_df = result_df.drop_duplicates()
result_df.sort_values('PValue', ascending = True, inplace = True)
result_df.drop(['Distance','PValue','length_distribution'],inplace=True,axis=1)
result_df.drop_duplicates(inplace=True) 
result_df.to_csv('common_positive_loci_7.csv',index=False)

#### For use of clearing all the output folders so you can rerun without errors and replace all the existing data

In [ ]:
import shutil
def clear_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.remove(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)


for folder_name in folders:
    #gather the directory and file id
    if "DRAGEN Analysis" in folder_name:
        file_id = folder_name.split()[0]
        dir = f"project/MSI_Analyses/{folder_name}/{file_id}"
    else:
        continue
    clear_folder(f'{dir}/{file_id}_outputs')
    
    print(f'done clearing {file_id}')
    
    
    
print('------------DONE---------------')

#### Creates the final table output containing comparison of protein mutation with distance/positivity of common loci

In [ ]:
# Load data
caseList_df = pd.read_csv('CaseList+Dir.csv')
MMR_df = pd.read_csv('Cases_with_MMR_CSV - Cases_with_MMR_CSV.csv')
caseList_df.rename(columns={'Case #':'Case'}, inplace=True)
combined_df = pd.merge(MMR_df, caseList_df, on='Case')

# Drop unnecessary columns
combined_df.drop(['Date Created', 'Laboratory', 'Req Patient/Source Name', 'Authorizing Provider', 'Case Status', 'Tasks', 'Case Type'], axis=1, inplace=True)

# Initialize loci columns with None
loci_list = ['BAT25', 'BAT26', 'NR-21', 'NR-24', 'NR-27']
for loci in loci_list:
    combined_df[loci] = None

# Remove front portion and rename
combined_df['SAMPLE_DIR_PATH'] = combined_df['SAMPLE_DIR_PATH'].apply(lambda x: x.split('/')[-1])
combined_df.rename(columns={'SAMPLE_DIR_PATH': 'file_id'}, inplace=True)

# Manually entered MMRIHC column
combined_df['MLH1'] = ['+','-','+','+','-','+','+','+','-','-']
combined_df['MSH2'] = ['-','-','-','-','-','-','-','-','-','-']
combined_df['MSH6'] = ['-','-','-','-','-','-','-','-','+','-']
combined_df['PMS2'] = ['+','-','+/-','+/-','-','+','+/-','+/-','-','-']

# Remove MMRIHC Column
combined_df.drop(['MMRIHC'], axis=1, inplace=True)

folders = os.listdir('project/MSI_Analyses')

def evaluate_loci(row):
    return row['Distance']

def get_data(row):
    original_file_id = row['file_id']
    modified_file_id = original_file_id.replace('_', '-')
    complete_path = None
    
    # find *_final.csv
    for folder_name in folders:
        if modified_file_id in folder_name:
            complete_path = f'project/MSI_Analyses/{folder_name}/{modified_file_id}/{modified_file_id}_outputs/{modified_file_id}_final.csv'
            break

    if complete_path and os.path.exists(complete_path):
        temp_final_df = pd.read_csv(complete_path)
        
        # Update each locus column for this specific file_id
        for loci in loci_list:
            matching_row = temp_final_df[temp_final_df['Target_Loci'] == loci]
            if not matching_row.empty:
                row[loci] = evaluate_loci(matching_row.iloc[0])
    
    return row

# Apply function
combined_df = combined_df.apply(get_data, axis=1)

# MSI status column 
def determine_msi_status(row):
    positive_count = sum(row[loci] is not None and row[loci] >= 0.1 for loci in loci_list)
    if positive_count >= 2:
        return 'MSI High'
    elif positive_count == 1:
        return 'MSI Low'
    else:
        return 'MSI Stable'

combined_df['MSI Status'] = combined_df.apply(determine_msi_status, axis=1)

# Drop unnecessary columns
combined_df.drop(['Code', 'Case', 'POSITIVE'], axis=1, inplace=True)
# print(combined_df.columns.tolist())
combined_df.rename(columns={'file_id':'SAMPLE_DIR_PATH'},inplace=True)
# Reorder columns
final_column_order = ['SAMPLE_DIR_PATH', 'MLH1', 'MSH2', 'MSH6', 'PMS2'] + loci_list + ['MSI Status']
combined_df = combined_df[final_column_order]

print(combined_df)
combined_df.to_csv('final_table.csv',index=False)


#### Creates a table which contains the total distances of all loci combined to achieve an "MSI Score" which is a total instability measure

In [ ]:
caseList_df = pd.read_csv('CaseList+Dir.csv')
caseList_df.drop(['Code','Case #','POSITIVE'],axis=1,inplace=True)
caseList_df['SAMPLE_DIR_PATH'] = caseList_df['SAMPLE_DIR_PATH'].apply(lambda x: x.split('/')[-1])
msi_score_df = pd.DataFrame()
msi_score_df['SAMPLE'] = caseList_df['SAMPLE_DIR_PATH']

def get_msi_score(row):
    original_file_id = row['SAMPLE']
    modified_file_id = original_file_id.replace('_', '-')
    complete_path = None
    
    #find *final.csv
    for folder_name in folders:
        if modified_file_id in folder_name:
            complete_path = f'project/MSI_Analyses/{folder_name}/{modified_file_id}/{modified_file_id}_outputs/{modified_file_id}_final.csv'
            break

    if complete_path and os.path.exists(complete_path):
        temp_final_df = pd.read_csv(complete_path)
        
        # Sum all values
        total_distance = temp_final_df['Distance'].sum()
        row['MSI Score'] = total_distance
    
    return row


msi_score_df = msi_score_df.apply(get_msi_score,axis=1)




print(msi_score_df)
msi_score_df.to_csv('MSI_Score_Table.csv',index=False)